# asyncio orchestration + multiprocessing.Pool (spawn) via `asyncio.to_thread`

The pool call is blocking, so it is run in a thread to avoid blocking the event loop.
Worker function is in an importable module to satisfy spawn pickling.


In [ ]:
from __future__ import annotations

from pathlib import Path
from tempfile import TemporaryDirectory
import importlib
import sys
import uuid

_temp_module_dir_ctx = TemporaryDirectory(
    prefix="nbast_pool_worker_",
    dir=str(Path.cwd()),
)
module_dir = Path(_temp_module_dir_ctx.name)
module_name = f"nbast_pool_worker_{uuid.uuid4().hex}"
module_path = module_dir / f"{module_name}.py"
module_path.write_text(
    "def cpu_fn(x):\n" "    return x + 10\n",
    encoding="utf-8",
)

if str(module_dir) not in sys.path:
    sys.path.insert(0, str(module_dir))
importlib.invalidate_caches()
wm = importlib.import_module(module_name)

In [ ]:
import asyncio
import multiprocessing as mp


async def main() -> None:
    ctx = mp.get_context("spawn")
    xs = [0, 1, 2, 3]
    with ctx.Pool(processes=2) as pool:
        # pool.map is blocking, so run it in a thread.
        ys = await asyncio.to_thread(pool.map, wm.cpu_fn, xs)
    assert ys == [x + 10 for x in xs]


try:
    await main()
finally:
    sys.modules.pop(module_name, None)
    if str(module_dir) in sys.path:
        sys.path.remove(str(module_dir))
    _temp_module_dir_ctx.cleanup()